In [ ]:
# Install required packages
install.packages(c("ggplot2", "forecast", "TSA", "tseries", "dplyr",
                   "knitr", "gridExtra", "MASS", "lawstat",
                   "FinTS", "ggthemes"))

In [ ]:
# Define the plot size
options(repr.plot.width = 24, repr.plot.height = 12)

In [ ]:
# Load required libraries
library(ggplot2)
library(forecast)
library(tseries)
library(TSA)
library(dplyr)
library(gridExtra)
library(knitr)
library(MASS)
library(lawstat) # For runs test
library(FinTS)   # For ARCH test

# ==========================================
# 1. DATA LOADING AND EXPLORATION
# ==========================================

In [ ]:
# Load the temperature data (avoid using 'ts' as variable name)
temp_data <- read.csv("../data/algiers_temp.csv")

In [ ]:
# Convert the Date column to Date format
temp_data$time <- as.Date(temp_data$time)

In [ ]:
# Display the first few rows
cat("First 5 rows of data:\n")
print(head(temp_data, 5))   

In [ ]:
# Display basic information about the dataset
cat("\nDataset info:\n")
str(temp_data)

In [ ]:
# Display summary statistics
cat("\nSummary statistics:\n")
print(summary(temp_data))

In [ ]:
# Create a proper time series object with daily frequency
temp_ts <- ts(temp_data$temperature, frequency = 365.25)


# ==========================================
# 2. TIME SERIES VISUALIZATION
# ==========================================

In [ ]:
# Plot the temperature data
p1 <- ggplot(temp_data, aes(x = time, y = temperature)) +
  geom_line(color = "blue") +
  labs(title = "Algiers Temperature Data (2002-2023)",
       x = "Date", y = "Temperature in Celsius") +
  theme_minimal() +
  theme(plot.title = element_text(hjust = 0.5))
print(p1)

In [ ]:
# Convert to monthly data for better seasonal analysis
monthly_temps <- aggregate(temperature ~ format(time, "%Y-%m"), temp_data, mean)
names(monthly_temps) <- c("yearmonth", "temperature")


In [ ]:
# Convert to time series object
monthly_temp_ts <- ts(monthly_temps$temperature, frequency = 12, 
                      start = c(2002, 1))

In [ ]:
# Perform decomposition
decomp_monthly <- decompose(monthly_temp_ts)

In [ ]:
# Create separate plots for each component
par(bg = "white")
par(mfrow = c(4, 1), mar = c(2, 4, 2, 1))
plot(monthly_temp_ts, main = "Original Series", ylab = "Temperature (°C)")
plot(decomp_monthly$trend, main = "Trend Component", ylab = "Trend")
plot(decomp_monthly$seasonal, main = "Seasonal Component", ylab = "Seasonal")
plot(decomp_monthly$random, main = "Random Component", ylab = "Random")
par(mfrow = c(1, 1))

# ==========================================
# 3. STATIONARITY TESTING
# ==========================================

In [ ]:
# Check for stationarity visually with ACF and PACF plots
par(bg = "white")
par(mfrow = c(1, 2))
acf(temp_ts, main = "ACF of Algiers Temperature", lag.max = 100)
pacf(temp_ts, main = "PACF of Algiers Temperature", lag.max = 100)
par(mfrow = c(1, 1))

In [ ]:
# Formal stationarity test
adf_test <- adf.test(temp_ts)
print("Augmented Dickey-Fuller Test for Stationarity:")
print(adf_test)

In [ ]:

# KPSS test for stationarity (alternative test)
kpss_test <- kpss.test(temp_ts)
print("KPSS Test for Stationarity:")
print(kpss_test)

In [ ]:
# Conclusion on stationarity
if (adf_test$p.value >= 0.05) {
  cat("ADF test: The series is NOT stationary (p >= 0.05)\n")
} else {
  cat("ADF test: The series is stationary (p < 0.05)\n")
}

if (kpss_test$p.value < 0.05) {
  cat("KPSS test: The series is NOT stationary (p < 0.05)\n")
} else {
  cat("KPSS test: The series is stationary (p >= 0.05)\n")
}

# ==========================================
# 4. DATA TRANSFORMATIONS
# ==========================================

In [ ]:
# Check if we need to stabilize variance
lambda <- BoxCox.lambda(monthly_temp_ts)
cat("\nOptimal lambda for Box-Cox transformation:", lambda, "\n")

In [ ]:
# Show Box-Cox transformation lambda selection plot
par(bg = "white")
boxcox(temp_ts ~ time(temp_ts),
       lambda = seq(-2, 2, by = 0.1),
       plotit = TRUE,
       main = "Box-Cox Transformation Parameter Selection")

In [ ]:
# Apply Box-Cox transformation
temp_boxcox <- BoxCox(temp_ts, lambda)
cat("Box-Cox transformation applied with lambda =", lambda, "\n")


In [ ]:
# Plot the transformed series
autoplot(temp_boxcox) +
  labs(title = "Transformed Algiers Temperature",
       x = "Time",
       y = "Transformed Temperature") +
  theme_minimal() +
  theme(plot.title = element_text(hjust = 0.5))

In [ ]:
# Apply seasonal differencing with monthly data (lag=12)
temp_seasonal_diff <- diff(temp_boxcox, lag = 12)

In [ ]:
# Plot the seasonally differenced series
autoplot(temp_seasonal_diff) +
  labs(title = "Seasonally Differenced Temperature",
       x = "Time",
       y = "Differenced Value") +
  theme_minimal() +
  theme(plot.title = element_text(hjust = 0.5))

In [ ]:
# Check stationarity of seasonally differenced series
adf_test_seasonal <- adf.test(temp_seasonal_diff)
print("ADF Test after seasonal differencing:")
print(adf_test_seasonal)

In [ ]:
# Conclusion on stationarity
if (adf_test_seasonal$p.value >= 0.05) {
  cat("ADF test: The series is NOT stationary (p >= 0.05)\n")
} else {
  cat("ADF test: The series is stationary (p < 0.05)\n")
}

In [ ]:
# Check ACF and PACF of differenced series
par(bg = "white")
par(mfrow = c(1, 2))
acf(bitcoin_d1, main = "ACF of Differenced Series", lag.max = 100)
pacf(bitcoin_d1, main = "PACF of Differenced Series", lag.max = 100)
par(mfrow = c(1, 1))

# ==========================================
# 5. MODEL SPECIFICATION AND SELECTION
# ==========================================

In [76]:
# Auto ARIMA for automatic model selection
auto_arima_model <- auto.arima(temp_boxcox, 
                               seasonal = TRUE,
                               stepwise = FALSE,
                               approximation = FALSE)
summary(auto_arima_model)

In [ ]:
# Plot the BIC values
par(bg = "white")
plot(arma_subset, legendtex = NULL,
     main = "BIC Values for Different ARMA Models")

In [ ]:
# Split the data for forecasting validation
n <- length(bitcoin_ts)
train_size <- n - 30  # Leave last 30 observations for testing

# Use direct subsetting instead of window()
bitcoin_train <- ts(bitcoin_ts[1:train_size], frequency = frequency(bitcoin_ts))
bitcoin_test <- ts(bitcoin_ts[(train_size + 1):n],
                   frequency = frequency(bitcoin_ts))

# Apply log transformation to training data
bitcoin_train_boxcox <- log(bitcoin_train)

# ==========================================
# 6. MODEL FITTING AND COMPARISON
# ==========================================

# ==========================================
# 7. DIAGNOSTIC CHECKING
# ==========================================

In [ ]:
# Standard diagnostic checking
checkresiduals(best_model)

In [ ]:
# Add more diagnostic tests for the best model
cat("\n======== Additional Diagnostic Tests for Best Model ========\n")

In [ ]:
# Extract residuals from the best model
residuals_best <- residuals(best_model)

In [ ]:
# Shapiro-Wilk test for normality
sw_test <- shapiro.test(residuals_best)
cat("\nShapiro-Wilk test for normality:\n")
print(sw_test)
if (sw_test$p.value < 0.05) {
  cat("Conclusion: Residuals are NOT normally distributed (p < 0.05)\n")
} else {
  cat("Conclusion: Residuals appear to be normally distributed (p >= 0.05)\n")
}

In [ ]:
# Runs test for randomness
runs_test <- runs.test(residuals_best)
cat("\nRuns test for randomness:\n")
print(runs_test)
if (runs_test$p.value < 0.05) {
  cat("Conclusion: Residuals are NOT random (p < 0.05)\n")
} else {
  cat("Conclusion: Residuals appear to be random (p >= 0.05)\n")
}

In [ ]:
# Ljung-Box test for autocorrelation (again, but with detailed output)
lb_test <- Box.test(residuals_best, lag = 20, type = "Ljung-Box")
cat("\nLjung-Box test for autocorrelation (lag=20):\n")
print(lb_test)
if (lb_test$p.value < 0.05) {
  cat("Conclusion: Residuals have significant autocorrelation (p < 0.05)\n")
} else {
  cat("Conclusion: No significant autocorrelation in residuals (p >= 0.05)\n")
}

In [ ]:
# McLeod-Li test for ARCH effects
mcleod_li_test <- Box.test(residuals_best^2, lag = 20, type = "Ljung-Box")
cat("\nMcLeod-Li test for ARCH effects:\n")
print(mcleod_li_test)
if (mcleod_li_test$p.value < 0.05) {
  cat("Conclusion: ARCH effects present in residuals (p < 0.05)\n")
} else {
  cat("Conclusion: No significant ARCH effects in residuals (p >= 0.05)\n")
}

# ==========================================
# 8. FORECASTING EVALUATION
# ==========================================


In [ ]:
# Forecast with the best model
forecast_horizon <- length(bitcoin_test)
forecasts <- forecast(best_model, h=forecast_horizon)

In [ ]:
# Inverse log transform forecasts
forecasts_original <- exp(forecasts$mean)

In [ ]:
# Combine with actual test data for comparison
forecast_comparison <- data.frame(
  Date = tail(bitcoin_data$Date, forecast_horizon),
  Actual = as.numeric(bitcoin_test),
  Forecast = as.numeric(forecasts_original)
)

In [ ]:
# Calculate forecast accuracy metrics manually
f_values <- as.numeric(forecasts_original)
a_values <- as.numeric(bitcoin_test)

In [ ]:
# Calculate accuracy metrics manually
me <- mean(f_values - a_values)
rmse <- sqrt(mean((f_values - a_values)^2))
mae <- mean(abs(f_values - a_values))
mape <- mean(abs((f_values - a_values)/a_values)) * 100

In [ ]:
# Create a data frame with the accuracy metrics
accuracy_metrics <- data.frame(
  ME = me,
  RMSE = rmse,
  MAE = mae,
  MAPE = mape
)

print("Forecast Accuracy Metrics:")
print(accuracy_metrics)


In [ ]:
# Plot forecasts vs actual values
p4 <- ggplot(forecast_comparison, aes(x = Date)) +
  geom_line(aes(y = Actual, color = "Actual"), size = 1) +
  geom_line(aes(y = Forecast, color = "Forecast"), size = 1) +
  labs(title = "Bitcoin Price: Forecast vs Actual",
       x = "Date", y = "Price (USD)") +
  scale_color_manual(values = c("Actual" = "blue", "Forecast" = "red")) +
  theme_minimal() +
  theme(plot.title = element_text(hjust = 0.5, face = "bold"),
        legend.title = element_blank())
print(p4)

In [ ]:
# 4. Forecast visualization with prediction intervals
# Create a forecast object with prediction intervals
forecast_full <- forecast(best_model, h = forecast_horizon)

# Convert to dataframe for ggplot
forecast_df <- data.frame(
  Time = 1:forecast_horizon,
  Forecast = InvBoxCox(forecast_full$mean, lambda),
  Lower80 = InvBoxCox(forecast_full$lower[,1], lambda),
  Upper80 = InvBoxCox(forecast_full$upper[,1], lambda),
  Lower95 = InvBoxCox(forecast_full$lower[,2], lambda),
  Upper95 = InvBoxCox(forecast_full$upper[,2], lambda),
  Actual = as.numeric(bitcoin_test)
)

In [ ]:
# Plot with ggplot2
forecast_plot <- ggplot(forecast_df, aes(x = Time)) +
  geom_ribbon(aes(ymin = Lower95, ymax = Upper95, fill = "95% PI"), alpha = 0.2) +
  geom_ribbon(aes(ymin = Lower80, ymax = Upper80, fill = "80% PI"), alpha = 0.3) +
  geom_line(aes(y = Forecast, color = "Forecast"), size = 1) +
  geom_line(aes(y = Actual, color = "Actual"), size = 1) +
  scale_fill_manual(name = "Prediction Intervals", 
                    values = c("95% PI" = "lightblue", "80% PI" = "skyblue")) +
  scale_color_manual(name = "Series", 
                     values = c("Forecast" = "blue", "Actual" = "red")) +
  labs(title = "Bitcoin Price Forecast with Prediction Intervals",
       x = "Days Ahead",
       y = "Price (USD)") +
  theme_minimal() +
  theme(legend.position = "right",
        plot.title = element_text(hjust = 0.5, size = 14, face = "bold"))

print(forecast_plot)